In [1]:
# Dependencies
import requests
import json
from config import gkey
import pandas as pd
import gmaps

## Store Part I results into DataFrame

In [2]:
weather_data = pd.read_csv('../WeatherPy/weather_data.csv')  
weather_data.head()

,Unnamed: 0,City Name,City ID,Country,Latitude,Longitude,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,Bonthe,2409914,SL,7.53,-12.51,79.30,80,100,14.29
1,1,Concarneau,3024035,FR,47.88,-3.92,67.95,88,75,10.29
2,2,Inongo,2315417,CD,-1.95,18.27,80.01,54,49,3.42
3,3,Ballina,2177069,AU,-28.87,153.57,60.49,88,90,12.75
4,4,Puerto Ayora,3652764,EC,-0.74,-90.35,68.00,94,100,7.00


## Humidity Heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [4]:
#Set the values for the humidity heatmap
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)
max_humidity = weather_data["Humidity (%)"].max()

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the DataFrame to find these ideal weather conditions:

Maximum temperature lower than 80 degrees but higher than 60.
Humidity less than 30%.
Wind speed less than 20 mph.

In [6]:
ideal_weather = (weather_data["Temp (F)"] > 60) & (weather_data["Temp (F)"] < 80) & \
    (weather_data["Humidity (%)"] < 30) & (weather_data["Wind Speed (mph)"] < 20)

ideal_weather = weather_data[ideal_weather]
ideal_weather

,Unnamed: 0,City Name,City ID,Country,Latitude,Longitude,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
75,75,Cape Town,3369157,ZA,-33.93,18.42,76.10,27,0,13.87
97,97,Maltahöhe,3355624,NaN,-24.83,16.98,71.01,20,0,14.12
100,100,Burgersdorp,1014653,ZA,-31.00,26.33,64.15,22,0,11.56
131,131,Mungwi,904241,ZM,-10.17,31.37,75.22,25,0,7.11
193,193,Plettenberg Bay,964712,ZA,-34.05,23.37,78.22,20,0,4.85
353,353,Taltal,3870243,CL,-25.40,-70.48,62.46,29,0,3.96
372,372,Yumen,1528998,CN,40.28,97.20,78.53,17,0,15.82
379,379,Kruisfontein,986717,ZA,-34.00,24.73,78.69,19,0,5.95
382,382,Menongue,3347353,AO,-14.66,17.69,77.86,17,0,7.78


## Hotel Map

In [7]:
#Use the Google Places API to find the nearest hotels

#Set url parameters
ideal_coordinates = ideal_weather[["Latitude", "Longitude"]].astype(float)

target_coordinates = ideal_coordinates.values.tolist()

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

In [8]:
#Loop through all coordinates for ideal locations & get a url string for nearby hotels

hotel_name = []

for x in target_coordinates:
#     print([x][0][1])
    
    base_url = (f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?\
location={[x][0][0]},{[x][0][1]}&radius={target_radius}&keyword={target_search}&key={gkey}")
    response = requests.get(base_url)
    hotel_data = response.json()
    try:
        hotel_name.append((hotel_data["results"][0]["name"]))
    except:
        hotel_name.append("(no hotels found)")
               

ideal_weather["Hotel Name"] = hotel_name
ideal_weather

C:\Users\user\anaconda3\envs\NewPythonData\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Unnamed: 0,City Name,City ID,Country,Latitude,Longitude,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
75,75,Cape Town,3369157,ZA,-33.93,18.42,76.10,27,0,13.87,The Bay Hotel
97,97,Maltahöhe,3355624,NaN,-24.83,16.98,71.01,20,0,14.12,Hotel Maltahöhe
100,100,Burgersdorp,1014653,ZA,-31.00,26.33,64.15,22,0,11.56,The Kingsman boutique hotel
131,131,Mungwi,904241,ZM,-10.17,31.37,75.22,25,0,7.11,(no hotels found)
193,193,Plettenberg Bay,964712,ZA,-34.05,23.37,78.22,20,0,4.85,Sky Villa Boutique Hotel
353,353,Taltal,3870243,CL,-25.40,-70.48,62.46,29,0,3.96,"Hotel- Galería ""Las brisas"""
372,372,Yumen,1528998,CN,40.28,97.20,78.53,17,0,15.82,(no hotels found)
379,379,Kruisfontein,986717,ZA,-34.00,24.73,78.69,19,0,5.95,(no hotels found)
382,382,Menongue,3347353,AO,-14.66,17.69,77.86,17,0,7.78,Ritz Lauca


In [10]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather.iterrows()]
locations = ideal_weather[["Latitude", "Longitude"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))